In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [9]:
df_national = gpd.read_file('../../Data/flood_ipc_seasonal_national.gpkg')
df_national = df_national.drop('level_0', 1)

In [6]:
lag_cols = df_national.columns.values[13:24]
lag_times = np.arange(1,4)

In [11]:
df_lagged = df_national.assign(**{
    '{}_t_{}'.format(col, t): df_national[col].shift(t)
    for t in lag_times
    for col in lag_cols
})

In [12]:
df_national['mean_ipc_prev'] = df_national.groupby(df_national['Name'])['mean_ipc'].shift(1)
df_national['mean_ipc_next'] = df_national.groupby(df_national['Name'])['mean_ipc'].shift(-1)
df_national['mean_ipc_nextnext'] = df_national.groupby(df_national['Name'])['mean_ipc'].shift(-2)
df_national['mean_ipc_nextnextnext'] = df_national.groupby(df_national['Name'])['mean_ipc'].shift(-3)

In [13]:
def baseline_model_table(df, y, cov_list, countries, groups = None):
    
    colnames = ['Country']
    for cov in cov_list:
        colnames.append('effect_size_{}'.format(cov))
        colnames.append('se_{}'.format(cov))
        colnames.append('p-value_{}'.format(cov))
    
    model_formula = '{} ~ '.format(y) + ' + '.join(cov_list)
    
    results_dict = {i : [] for i in colnames}
    
    for country in countries:
        df_select = df.loc[df['Name'] == country].reset_index()
        
        if groups == None:
            model = smf.ols(model_formula, data = df_select,
                       missing = 'drop').fit()
        else:
            model = smf.mixedlm(model_formula, data = df_select, groups = groups,
                               missing = 'drop').fit()
        
        effects = [model.params[j] for j in range(1, len(cov_list) + 1)]
        bses = [model.bse[j] for j in range(1, len(cov_list) + 1)]
        pvalues = [model.pvalues[j] for j in range(1, len(cov_list) + 1)]
        
        
        results_dict['Country'].append(country)


        for j, cov in enumerate(cov_list):

            results_dict['effect_size_{}'.format(cov)].append(effects[j])
            results_dict['se_{}'.format(cov)].append(bses[j])
            results_dict['p-value_{}'.format(cov)].append(pvalues[j])

    
    results_table = pd.DataFrame(data = results_dict, columns = colnames)
    return(results_table)

In [14]:
def sensitivity_analysis_seasons(df, y, cov_list, country, groups = None):
    
    colnames = ['Country', 'Year', 'Month']
    for cov in cov_list:
        colnames.append('effect_size_{}'.format(cov))
        colnames.append('se_{}'.format(cov))
        colnames.append('p-value_{}'.format(cov))
    
    model_formula = '{} ~ '.format(y) + ' + '.join(cov_list)
    
    df_select = df.loc[df['Name'] == country].reset_index()
    
    results_dict = {i : [] for i in colnames}
    
    for i, row in df_select.iterrows():
        df_select_loo = df_select.drop(i, axis = 0)
        
        if groups == None:
            model = smf.ols(model_formula, data = df_select_loo,
                       missing = 'drop').fit()
        else:
            model = smf.mixedlm(model_formula, data = df_select_loo, groups = groups,
                               missing = 'drop').fit()
        
        effects = [model.params[j] for j in range(1, len(cov_list) + 1)]
        bses = [model.bse[j] for j in range(1, len(cov_list) + 1)]
        pvalues = [model.pvalues[j] for j in range(1, len(cov_list) + 1)]
        
        
        results_dict['Country'].append(row['Name'])
        results_dict['Year'].append(row['Year'])
        results_dict['Month'].append(row['Month'])


        for j, cov in enumerate(cov_list):

            results_dict['effect_size_{}'.format(cov)].append(effects[j])
            results_dict['se_{}'.format(cov)].append(bses[j])
            results_dict['p-value_{}'.format(cov)].append(pvalues[j])

    
    results_table = pd.DataFrame(data = results_dict, 
                                 columns = colnames)
    return(results_table)

## Mixed effects models for all seasons

In [11]:
model0 = smf.mixedlm('mean_ipc ~ mean_ipc_prev + flood_1_7_days + flood_8_14_days + flood_15_21_days + flood_22_28_days + flood_29_plus_days',
                    data = df_national, groups = "Name", missing = "drop").fit()
model0.summary()

/Users/connor/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/connor/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2202: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
=============================================================
Model:               MixedLM   Dependent Variable:   mean_ipc
No. Observations:    599       Method:               REML    
No. Groups:          16        Scale:                0.0624  
Min. group size:     36        Log-Likelihood:       -53.5495
Max. group size:     38        Converged:            Yes     
Mean group size:     37.4                                    
-------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept           0.521    0.069  7.565 0.000  0.386  0.656
mean_ipc_prev       0.674    0.039 17.325 0.000  0.598  0.750
flood_1_7_days     -0.053    0.029 -1.832 0.067 -0.110  0.004
flood_8_14_days    -0.064    0.036 -1.792 0.073 -0.135  0.006
flood_15_21_days   -0.120    0.043 -2.797 0.005 -0.204 -0.036
flood_22_28_days   -0.113    0.048 -2.344 0.019 -0.208 -0.019
flood_29_plus_days -0.135    0.032 -4.188 0.000 -0.198 -0.072
Name Var            0.021    0.095                           
=============================================================

"""

In [12]:
model1 = smf.mixedlm('mean_ipc_next ~ mean_ipc + flood_1_7_days + flood_8_14_days + flood_15_21_days + flood_22_28_days + flood_29_plus_days',
                    data = df_national, groups = "Name", missing = "drop").fit()
model1.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
=============================================================
Model:              MixedLM Dependent Variable: mean_ipc_next
No. Observations:   599     Method:             REML         
No. Groups:         16      Scale:              0.0670       
Min. group size:    36      Log-Likelihood:     -70.4483     
Max. group size:    38      Converged:          Yes          
Mean group size:    37.4                                     
-------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept           0.525    0.060  8.798 0.000  0.408  0.642
mean_ipc            0.649    0.034 19.357 0.000  0.583  0.715
flood_1_7_days      0.067    0.031  2.121 0.034  0.005  0.128
flood_8_14_days    -0.008    0.041 -0.200 0.842 -0.089  0.072
flood_15_21_days   -0.041    0.046 -0.894 0.371 -0.132  0.049
flood_22_28_days   -0.045    0.044 -1.024 0.306 -0.132  0.041
flood_29_plus_days -0.059    0.034 -1.724 0.085 -0.127  0.008
Name Var            0.013    0.024                           
=============================================================

"""

In [13]:
model2 = smf.mixedlm('mean_ipc_nextnext ~ mean_ipc_next + flood_1_7_days + flood_8_14_days + flood_15_21_days + flood_22_28_days + flood_29_plus_days',
                    data = df_national, groups = "Name", missing = "drop").fit()
model2.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
             Mixed Linear Model Regression Results
===============================================================
Model:            MixedLM Dependent Variable: mean_ipc_nextnext
No. Observations: 583     Method:             REML             
No. Groups:       16      Scale:              0.0662           
Min. group size:  35      Log-Likelihood:     -64.0190         
Max. group size:  37      Converged:          Yes              
Mean group size:  36.4                                         
---------------------------------------------------------------
                      Coef. Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------
Intercept             0.451    0.060  7.469 0.000  0.333  0.569
mean_ipc_next         0.678    0.035 19.154 0.000  0.609  0.748
flood_1_7_days        0.057    0.032  1.786 0.074 -0.006  0.121
flood_8_14_days       0.072    0.044  1.652 0.098 -0.013  0.158
flood_15_21_days      0.078    0.047  1.668 0.095 -0.014  0.169
flood_22_28_days      0.100    0.045  2.216 0.027  0.012  0.188
flood_29_plus_days    0.099    0.035  2.854 0.004  0.031  0.166
Name Var              0.010    0.021                           
===============================================================

"""

In [14]:
model3 = smf.mixedlm('mean_ipc_nextnextnext ~ mean_ipc_nextnext + flood_1_7_days + flood_8_14_days + flood_15_21_days + flood_22_28_days + flood_29_plus_days',
                    data = df_national, groups = "Name", missing = "drop").fit()
model3.summary()

/Users/connor/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/connor/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2202: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/Users/connor/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/connor/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2202: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
/Users/connor/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/connor/opt/anaconda

<class 'statsmodels.iolib.summary2.Summary'>
"""
               Mixed Linear Model Regression Results
===================================================================
Model:            MixedLM Dependent Variable: mean_ipc_nextnextnext
No. Observations: 567     Method:             REML                 
No. Groups:       16      Scale:              0.0685               
Min. group size:  34      Log-Likelihood:     -71.9368             
Max. group size:  36      Converged:          No                   
Mean group size:  35.4                                             
--------------------------------------------------------------------
                     Coef.   Std.Err.    z     P>|z|  [0.025  0.975]
--------------------------------------------------------------------
Intercept             0.499     0.060   8.375  0.000   0.382   0.616
mean_ipc_nextnext     0.659     0.036  18.413  0.000   0.589   0.729
flood_1_7_days       -0.006     0.033  -0.194  0.846  -0.071   0.058
flood_8_14_days       0.076     0.042   1.809  0.070  -0.006   0.158
flood_15_21_days      0.009     0.050   0.175  0.861  -0.090   0.107
flood_22_28_days      0.032     0.046   0.694  0.488  -0.058   0.122
flood_29_plus_days    0.002     0.036   0.055  0.956  -0.068   0.072
Name Var              0.010     0.016                               
===================================================================

"""

# Identifying countries with internally significant flood responses across all seasons

In [15]:
country_names = df_national.Name.unique()

In [16]:
covariates_0 = ['mean_ipc_prev', 'flood_1_7_days', 'flood_8_14_days', 'flood_15_21_days', 'flood_22_28_days', 'flood_29_plus_days']
covariates_1 = ['mean_ipc', 'flood_1_7_days', 'flood_8_14_days', 'flood_15_21_days', 'flood_22_28_days', 'flood_29_plus_days']
covariates_2 = ['mean_ipc_next', 'flood_1_7_days', 'flood_8_14_days', 'flood_15_21_days', 'flood_22_28_days', 'flood_29_plus_days']
covariates_3 = ['mean_ipc_nextnext', 'flood_1_7_days', 'flood_8_14_days', 'flood_15_21_days', 'flood_22_28_days', 'flood_29_plus_days']

baseline_0 = baseline_model_table(df_national, 'mean_ipc', covariates_0, country_names)
baseline_1 = baseline_model_table(df_national, 'mean_ipc_next', covariates_1, country_names)
baseline_2 = baseline_model_table(df_national, 'mean_ipc_nextnext', covariates_2, country_names)
baseline_3 = baseline_model_table(df_national, 'mean_ipc_nextnextnext', covariates_3, country_names)

In [115]:
cols = np.array([col for col in baseline_0 if col.startswith('p-value_flood')])
time0_sig = baseline_0.loc[(baseline_0[cols] < 0.05).any(axis = 1)]
time0_sig_countries = time0_sig.Country.unique()
time0_tuples = [tuple((country, *cols[(time0_sig[time0_sig.Country == country][cols] < 0.05).any(axis = 0).values])) for country in time0_sig_countries]

print(time0_tuples)
time0_sig

[('Ethiopia', 'p-value_flood_15_21_days', 'p-value_flood_22_28_days'), ('Kenya', 'p-value_flood_29_plus_days'), ('Mozambique', 'p-value_flood_1_7_days'), ('Malawi', 'p-value_flood_8_14_days'), ('Sudan', 'p-value_flood_1_7_days')]


,Country,effect_size_mean_ipc_prev,se_mean_ipc_prev,p-value_mean_ipc_prev,effect_size_flood_1_7_days,se_flood_1_7_days,p-value_flood_1_7_days,effect_size_flood_8_14_days,se_flood_8_14_days,p-value_flood_8_14_days,effect_size_flood_15_21_days,se_flood_15_21_days,p-value_flood_15_21_days,effect_size_flood_22_28_days,se_flood_22_28_days,p-value_flood_22_28_days,effect_size_flood_29_plus_days,se_flood_29_plus_days,p-value_flood_29_plus_days
1,Ethiopia,0.684122,0.093585,3.158840e-08,-0.061466,0.111033,0.583836,-0.112128,0.069136,0.114963,-1.670268e-01,6.217290e-02,0.011500,-0.213387,0.100142,0.041140,-0.094538,0.090491,0.304230
2,Kenya,0.643132,0.155551,2.509901e-04,-0.027206,0.089091,0.762120,-0.158908,0.119790,0.194345,-9.818848e-02,1.074307e-01,0.367787,0.073246,0.154602,0.638984,-0.252790,0.102744,0.019653
4,Mozambique,0.709116,0.112890,6.365351e-07,0.183003,0.079358,0.028199,-0.184671,0.125232,0.150729,-1.223864e-17,1.780906e-17,0.497226,0.016488,0.132458,0.901768,-0.070992,0.061145,0.254776
6,Malawi,0.646842,0.122332,9.414010e-06,0.091640,0.068325,0.189588,-0.192798,0.071472,0.011197,0.000000e+00,0.000000e+00,NaN,0.000000,0.000000,NaN,-0.064437,0.084015,0.448905
9,Sudan,0.646785,0.159784,3.197246e-04,-0.208576,0.078109,0.011959,0.038372,0.135394,0.778747,-3.377162e-01,1.737165e-01,0.061011,-0.095502,0.075569,0.215731,-0.133818,0.077769,0.095270


In [121]:
cols = np.array([col for col in baseline_1 if col.startswith('p-value_flood')])
time1_sig = baseline_1.loc[(baseline_1[cols] < 0.05).any(axis = 1)]
time1_sig_countries = time1_sig.Country.unique()
time1_tuples = [tuple((country, *cols[(time1_sig[time1_sig.Country == country][cols] < 0.05).any(axis = 0).values])) for country in time1_sig_countries]

print(time1_tuples)
time1_sig

[('Mozambique', 'p-value_flood_29_plus_days'), ('Uganda', 'p-value_flood_1_7_days'), ('Zambia', 'p-value_flood_8_14_days')]


,Country,effect_size_mean_ipc,se_mean_ipc,p-value_mean_ipc,effect_size_flood_1_7_days,se_flood_1_7_days,p-value_flood_1_7_days,effect_size_flood_8_14_days,se_flood_8_14_days,p-value_flood_8_14_days,effect_size_flood_15_21_days,se_flood_15_21_days,p-value_flood_15_21_days,effect_size_flood_22_28_days,se_flood_22_28_days,p-value_flood_22_28_days,effect_size_flood_29_plus_days,se_flood_29_plus_days,p-value_flood_29_plus_days
4,Mozambique,0.702774,0.110634,5.231501e-07,0.191736,0.096069,0.055102,-0.072624,0.178967,0.687771,-1.316022e-16,1.909327e-16,0.495957,0.146475,0.127921,0.261246,-0.137029,0.061986,0.034829
13,Uganda,0.235113,0.164626,1.632426e-01,0.256334,0.099077,0.014590,-0.102429,0.107168,0.346578,-5.527967e-02,1.379591e-01,0.691392,0.004018,0.166499,0.980900,-0.157093,0.141258,0.274645
14,Zambia,0.921345,0.059171,1.730521e-16,0.019736,0.083118,0.813822,0.250384,0.059285,0.000186,-4.435121e-02,1.147785e-01,0.701753,0.000000,0.000000,NaN,0.031332,0.123037,0.800617


In [122]:
cols = np.array([col for col in baseline_2 if col.startswith('p-value_flood')])
time2_sig = baseline_2.loc[(baseline_2[cols] < 0.05).any(axis = 1)]
time2_sig_countries = time2_sig.Country.unique()
time2_tuples = [tuple((country, *cols[(time2_sig[time2_sig.Country == country][cols] < 0.05).any(axis = 0).values])) for country in time2_sig_countries]

print(time2_tuples)
time2_sig

[('Mali', 'p-value_flood_29_plus_days'), ('Malawi', 'p-value_flood_1_7_days'), ('Niger', 'p-value_flood_22_28_days')]


,Country,effect_size_mean_ipc_next,se_mean_ipc_next,p-value_mean_ipc_next,effect_size_flood_1_7_days,se_flood_1_7_days,p-value_flood_1_7_days,effect_size_flood_8_14_days,se_flood_8_14_days,p-value_flood_8_14_days,effect_size_flood_15_21_days,se_flood_15_21_days,p-value_flood_15_21_days,effect_size_flood_22_28_days,se_flood_22_28_days,p-value_flood_22_28_days,effect_size_flood_29_plus_days,se_flood_29_plus_days,p-value_flood_29_plus_days
3,Mali,0.412832,0.137243,0.005182,0.275678,0.193528,0.164297,3.850127e-16,3.715336e-16,0.308088,0.179893,0.238072,0.455580,0.306201,0.170505,0.082271,0.273714,0.099508,0.009839
6,Malawi,0.452264,0.137019,0.002494,0.319424,0.131686,0.021508,-3.546707e-02,1.191131e-01,0.767939,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.068833,0.086143,0.430537
7,Niger,0.503963,0.112637,0.000102,0.228128,0.124762,0.077432,2.220867e-01,2.161159e-01,0.312340,0.175029,0.101097,0.093675,0.411950,0.092745,0.000112,0.118425,0.084821,0.172909


In [123]:
cols = np.array([col for col in baseline_3 if col.startswith('p-value_flood')])
time3_sig = baseline_3.loc[(baseline_3[cols] < 0.05).any(axis = 1)]
time3_sig_countries = time3_sig.Country.unique()
time3_tuples = [tuple((country, *cols[(time3_sig[time3_sig.Country == country][cols] < 0.05).any(axis = 0).values])) for country in time3_sig_countries]

print(time3_tuples)
time3_sig

[('Mozambique', 'p-value_flood_1_7_days'), ('Mauritania', 'p-value_flood_22_28_days'), ('Malawi', 'p-value_flood_1_7_days'), ('Niger', 'p-value_flood_15_21_days'), ('Nigeria', 'p-value_flood_22_28_days'), ('Somalia', 'p-value_flood_8_14_days'), ('Uganda', 'p-value_flood_1_7_days'), ('Zambia', 'p-value_flood_1_7_days')]


,Country,effect_size_mean_ipc_nextnext,se_mean_ipc_nextnext,p-value_mean_ipc_nextnext,effect_size_flood_1_7_days,se_flood_1_7_days,p-value_flood_1_7_days,effect_size_flood_8_14_days,se_flood_8_14_days,p-value_flood_8_14_days,effect_size_flood_15_21_days,se_flood_15_21_days,p-value_flood_15_21_days,effect_size_flood_22_28_days,se_flood_22_28_days,p-value_flood_22_28_days,effect_size_flood_29_plus_days,se_flood_29_plus_days,p-value_flood_29_plus_days
4,Mozambique,0.967649,0.128824,3.506940e-08,-0.320984,0.123840,0.014996,0.095280,0.138308,0.496555,-1.746688e-16,2.237368e-16,0.441536,0.028789,0.137553,0.835734,0.044563,0.065465,0.501636
5,Mauritania,0.286069,0.149533,6.532436e-02,0.016642,0.247755,0.946891,0.000000,0.000000,NaN,0.000000e+00,0.000000e+00,NaN,0.699790,0.258324,0.011044,0.000000,0.000000,NaN
6,Malawi,0.914150,0.151073,1.384744e-06,-0.422335,0.131365,0.003194,0.069405,0.072792,0.348230,0.000000e+00,0.000000e+00,NaN,0.000000,0.000000,NaN,-0.007969,0.093890,0.932942
7,Niger,0.286585,0.147781,6.225409e-02,0.178657,0.123967,0.160249,-0.118879,0.224035,0.599717,2.213331e-01,1.063713e-01,0.046396,0.251762,0.123593,0.050867,-0.122642,0.090851,0.187487
8,Nigeria,0.163505,0.165463,3.312436e-01,0.030782,0.057150,0.594257,-0.029502,0.118718,0.805495,3.007207e-02,2.131722e-01,0.888790,0.313653,0.150617,0.046229,-0.124755,0.110751,0.269213
10,Somalia,0.760976,0.100963,2.105031e-08,-0.201704,0.260147,0.444204,0.250900,0.109433,0.029049,-4.099664e-02,2.615857e-01,0.876513,0.000000,0.000000,NaN,0.526351,0.260450,0.052290
13,Uganda,0.179255,0.161852,2.771684e-01,0.296143,0.097421,0.004978,-0.045286,0.105926,0.672153,-8.300839e-02,1.350226e-01,0.543498,-0.207413,0.166273,0.222223,-0.070526,0.144135,0.628307
14,Zambia,1.044982,0.075194,7.351859e-15,-0.258208,0.100276,0.015020,0.031106,0.075493,0.683151,-9.335861e-03,1.398899e-01,0.947219,0.000000,0.000000,NaN,0.000000,0.000000,NaN


# Identifying floods & seasons of interest within above countries

### Time 0

In [ ]:
sub_dfs = []

for country in country_names:
    sub_dfs.append(sensitivity_analysis_seasons(df_national, 'mean_ipc', covariates_0, country))

time_0_models = pd.concat(sub_dfs, ignore_index = True)
    
time_0_bigseasons = time_0_models.loc[time_0_models.Country.isin(time0_sig_countries)]

In [106]:
for tup in time0_tuples:
    x = time_0_bigseasons.loc[((time_0_bigseasons.Country == tup[0]) & (time_0_bigseasons[[*tup[1:]]] >= 0.05).any(axis = 1))]
    print(tup[0])
    print("\n")
    print(x[['Country', 'Year', 'Month', *tup[1:]]])
    print("\n")

Ethiopia


     Country  Year  Month  p-value_flood_15_21_days  p-value_flood_22_28_days
47  Ethiopia  2011      7                  0.010356                  0.053231
52  Ethiopia  2012     10                  0.009553                  0.634544


Kenya


    Country  Year  Month  p-value_flood_29_plus_days
104   Kenya  2016      2                    0.062636
117   Kenya  2020      2                    0.078647


Mozambique


        Country  Year  Month  p-value_flood_1_7_days
183  Mozambique  2016      6                0.086245
186  Mozambique  2017      6                0.187331
193  Mozambique  2019      6                0.065291


Malawi


    Country  Year  Month  p-value_flood_8_14_days
264  Malawi  2017      6                 0.203365


Sudan


Empty DataFrame
Columns: [Country, Year, Month, p-value_flood_1_7_days]
Index: []




### Time 1

In [101]:
sub_dfs = []

for country in country_names:
    sub_dfs.append(sensitivity_analysis_seasons(df_national, 'mean_ipc_next', covariates_1, country))

time_1_models = pd.concat(sub_dfs, ignore_index = True)
    
time_1_bigseasons = time_1_models.loc[time_1_models.Country.isin(time1_sig_countries)]

In [105]:
for tup in time1_tuples:
    x = time_1_bigseasons.loc[((time_1_bigseasons.Country == tup[0]) & (time_1_bigseasons[[*tup[1:]]] >= 0.05).any(axis = 1))]
    print(tup[0])
    print("\n")
    print(x[['Country', 'Year', 'Month', *tup[1:]]])
    print("\n")

Mozambique


        Country  Year  Month  p-value_flood_29_plus_days
159  Mozambique  2010      4                    0.054625
183  Mozambique  2016      6                    0.053674
185  Mozambique  2017      2                    0.434675
192  Mozambique  2019      2                    0.061583


Uganda


    Country  Year  Month  p-value_flood_1_7_days
540  Uganda  2018      6                0.386194


Zambia


Empty DataFrame
Columns: [Country, Year, Month, p-value_flood_8_14_days]
Index: []




### Time 2

In [107]:
sub_dfs = []

for country in country_names:
    sub_dfs.append(sensitivity_analysis_seasons(df_national, 'mean_ipc_nextnext', covariates_2, country))

time_2_models = pd.concat(sub_dfs, ignore_index = True)
    
time_2_bigseasons = time_2_models.loc[time_2_models.Country.isin(time2_sig_countries)]

In [108]:
for tup in time2_tuples:
    x = time_2_bigseasons.loc[((time_2_bigseasons.Country == tup[0]) & (time_2_bigseasons[[*tup[1:]]] >= 0.05).any(axis = 1))]
    print(tup[0])
    print("\n")
    print(x[['Country', 'Year', 'Month', *tup[1:]]])
    print("\n")

Mali


    Country  Year  Month  p-value_flood_29_plus_days
126    Mali  2011     10                    0.091558


Malawi


    Country  Year  Month  p-value_flood_1_7_days
261  Malawi  2016      6                0.127801
262  Malawi  2016     10                0.157870


Niger


    Country  Year  Month  p-value_flood_22_28_days
274   Niger  2009     10                  0.065509




### Time 3

In [112]:
sub_dfs = []

for country in country_names:
    sub_dfs.append(sensitivity_analysis_seasons(df_national, 'mean_ipc_nextnextnext', covariates_3, country))

time_3_models = pd.concat(sub_dfs, ignore_index = True)
    
time_3_bigseasons = time_3_models.loc[time_3_models.Country.isin(time3_sig_countries)]

In [113]:
for tup in time3_tuples:
    x = time_3_bigseasons.loc[((time_3_bigseasons.Country == tup[0]) & (time_3_bigseasons[[*tup[1:]]] >= 0.05).any(axis = 1))]
    print(tup[0])
    print("\n")
    print(x[['Country', 'Year', 'Month', *tup[1:]]])
    print("\n")

Mozambique


        Country  Year  Month  p-value_flood_1_7_days
183  Mozambique  2016      6                0.236177


Mauritania


Empty DataFrame
Columns: [Country, Year, Month, p-value_flood_22_28_days]
Index: []


Malawi


    Country  Year  Month  p-value_flood_1_7_days
261  Malawi  2016      6                0.111244


Niger


    Country  Year  Month  p-value_flood_15_21_days
275   Niger  2010      1                  0.052402
283   Niger  2012      1                  0.054962
286   Niger  2012     10                  0.050063
288   Niger  2013      4                  0.050477
291   Niger  2014      1                  0.050713
292   Niger  2014      4                  0.050453
295   Niger  2015      1                  0.052101
297   Niger  2015      7                  0.050604
299   Niger  2016      2                  0.050589
302   Niger  2017      2                  0.050683
304   Niger  2017     10                  0.114888
305   Niger  2018      2                  0.062680


# New autoregressive lagged approach

In [21]:
covariates = np.array([col for col in df_lagged.columns.values if (col.startswith('flood_') | col.startswith('mean_ipc_'))])

In [23]:
baseline_new = baseline_model_table(df_lagged, 'mean_ipc', covariates, country_names)

In [28]:
cols = np.array([col for col in baseline_new if col.startswith('p-value_flood')])
lag_sig = baseline_new.loc[(baseline_new[cols] < 0.05).any(axis = 1)]
lag_sig_countries = lag_sig.Country.unique()
lag_tuples = [tuple((country, *cols[(lag_sig[lag_sig.Country == country][cols] < 0.05).any(axis = 0).values])) for country in lag_sig_countries]

for tup in lag_tuples:
    print(tup)
    print("\n")
lag_sig

('Burkina Faso', 'p-value_flood_8_14_days', 'p-value_flood_8_14_days_t_1', 'p-value_flood_22_28_days_t_1', 'p-value_flood_22_28_days_t_3', 'p-value_flood_29_plus_days_t_3')


('Ethiopia', 'p-value_flood_15_21_days')


('Kenya', 'p-value_flood_15_21_days', 'p-value_flood_29_plus_days', 'p-value_flood_8_14_days_t_2', 'p-value_flood_8_14_days_t_3', 'p-value_flood_15_21_days_t_3', 'p-value_flood_22_28_days_t_3')


('Mali', 'p-value_flood_29_plus_days_t_1')


('Mozambique', 'p-value_flood_22_28_days_t_3', 'p-value_flood_29_plus_days_t_3')


('Mauritania', 'p-value_flood_8_14_days_t_2')


('Malawi', 'p-value_flood_15_21_days', 'p-value_flood_1_7_days_t_1', 'p-value_flood_8_14_days_t_1', 'p-value_flood_15_21_days_t_1', 'p-value_flood_1_7_days_t_2', 'p-value_flood_29_plus_days_t_3')


('Somalia', 'p-value_flood_8_14_days', 'p-value_flood_22_28_days_t_3')


('South Sudan', 'p-value_flood_15_21_days_t_3')


('Uganda', 'p-value_flood_8_14_days', 'p-value_flood_22_28_days', 'p-value_flood_29_plus_

,Country,effect_size_flood_1_7_days,se_flood_1_7_days,p-value_flood_1_7_days,effect_size_flood_8_14_days,se_flood_8_14_days,p-value_flood_8_14_days,effect_size_flood_15_21_days,se_flood_15_21_days,p-value_flood_15_21_days,...,p-value_flood_15_21_days_t_3,effect_size_flood_22_28_days_t_3,se_flood_22_28_days_t_3,p-value_flood_22_28_days_t_3,effect_size_flood_29_plus_days_t_3,se_flood_29_plus_days_t_3,p-value_flood_29_plus_days_t_3,effect_size_mean_ipc_t_3,se_mean_ipc_t_3,p-value_mean_ipc_t_3
0,Burkina Faso,8.454974e-17,9.919488e-17,0.405213,1.495200e-16,6.672669e-17,0.037890,-2.566061e-02,4.057846e-02,0.535096,...,0.095189,2.889827e-01,9.518397e-02,0.007105,0.317409,0.073065,0.000391,-0.184696,0.178340,0.314085
1,Ethiopia,-2.771645e-01,4.606778e-01,0.554913,2.942330e-02,1.400664e-01,0.835975,-3.378242e-01,1.494768e-01,0.036452,...,0.364466,0.000000e+00,0.000000e+00,NaN,0.272035,0.440234,0.544360,0.099619,0.435200,0.821524
2,Kenya,8.899322e-02,8.457692e-02,0.308333,-1.876188e-01,2.024653e-01,0.367865,-7.712798e-01,2.276396e-01,0.003753,...,0.017081,1.570440e+00,4.953706e-01,0.005938,-0.054192,0.271618,0.844373,-0.117073,0.075361,0.139859
3,Mali,-1.579966e-01,5.345550e-01,0.770767,-3.759635e-16,1.113767e-15,0.739395,1.643062e-01,1.088755e-01,0.147719,...,0.147719,5.061146e-02,9.477902e-02,0.599538,0.021406,0.424692,0.960327,1.070220,0.335198,0.004791
4,Mozambique,1.814217e-01,1.655257e-01,0.289288,-2.228840e-01,2.380672e-01,0.363075,-8.816760e-16,9.025579e-16,0.343176,...,0.510170,1.291713e+00,5.305731e-01,0.026993,0.772237,0.312834,0.025220,0.360621,0.347138,0.314324
5,Mauritania,1.022281e-01,1.176217e-01,0.395625,-5.398298e-16,3.431358e-16,0.132171,-5.414443e-17,6.891140e-17,0.441726,...,0.770414,8.673834e-02,1.079743e-01,0.431723,0.047130,0.070806,0.513653,-0.097318,0.097252,0.329559
6,Malawi,9.763613e-02,6.697632e-02,0.159697,-6.796818e-03,7.007474e-02,0.923651,6.786426e-17,2.926402e-17,0.030558,...,0.498776,-5.833363e-02,7.322869e-02,0.434600,0.067344,0.029666,0.033855,-0.061126,0.122062,0.621738
10,Somalia,2.011423e-01,4.330721e-01,0.648579,-4.086595e-01,1.917110e-01,0.048887,-6.122357e-01,4.395870e-01,0.182747,...,0.298435,1.035727e+00,4.400585e-01,0.031708,0.840375,0.404844,0.054393,0.899345,0.319144,0.012373
11,South Sudan,-1.570321e-01,2.506864e-01,0.539882,-8.439528e-02,2.307719e-01,0.719373,-1.625051e-01,1.610316e-01,0.327921,...,0.048734,4.619899e-01,4.697945e-01,0.340055,-0.383899,0.271863,0.177073,-0.119463,0.277946,0.673068
13,Uganda,1.078965e-02,4.160559e-02,0.798494,-1.250037e-01,4.469089e-02,0.012381,5.655162e-02,6.141164e-02,0.370007,...,0.022973,1.854211e-17,9.000782e-18,0.055042,-0.013653,0.063930,0.833425,-0.113463,0.042207,0.015552


In [30]:
sub_dfs = []

for country in country_names:
    sub_dfs.append(sensitivity_analysis_seasons(df_lagged, 'mean_ipc', covariates, country))

new_models = pd.concat(sub_dfs, ignore_index = True)
    
new_bigseasons = new_models.loc[new_models.Country.isin(lag_sig_countries)]

In [31]:
for tup in lag_tuples:
    x = new_bigseasons.loc[((new_bigseasons.Country == tup[0]) & (new_bigseasons[[*tup[1:]]] >= 0.05).any(axis = 1))]
    print(tup[0])
    print("\n")
    print(x[['Country', 'Year', 'Month', *tup[1:]]])
    print("\n")

Burkina Faso


         Country  Year  Month  p-value_flood_8_14_days  \
3   Burkina Faso  2010      4                 0.000928   
5   Burkina Faso  2010     10                 0.098005   
6   Burkina Faso  2011      1                 0.008880   
8   Burkina Faso  2011      7                 0.076028   
10  Burkina Faso  2012      1                 0.000037   
11  Burkina Faso  2012      4                 0.621840   
12  Burkina Faso  2012      7                 0.102094   
13  Burkina Faso  2012     10                 0.032665   
14  Burkina Faso  2013      1                 0.502874   
15  Burkina Faso  2013      4                 0.109381   
16  Burkina Faso  2013      7                 0.242439   
17  Burkina Faso  2013     10                 0.000146   
18  Burkina Faso  2014      1                 0.030332   
20  Burkina Faso  2014      7                 0.058388   
21  Burkina Faso  2014     10                 0.770942   
22  Burkina Faso  2015      1                 0.079301   